In [ ]:
import pandas as pd

In [ ]:
dataset_train1 = pd.read_csv('../input/tabular-playground-series-sep-2021/train.csv', index_col='id')
dataset_test1 = pd.read_csv('../input/tabular-playground-series-sep-2021/test.csv', index_col='id')
y = dataset_train1.claim
y = pd.DataFrame(y)
dataset_train = dataset_train1.drop(['claim'], axis=1)

In [ ]:
#

In [ ]:
dataset_train['nan_count'] = dataset_train.isnull().sum(axis=1)
#dataset_train['nan_count']  = dataset_train['nan_count']/dataset_train['nan_count'].max()

dataset_train['min_row'] = dataset_train.min(axis=1)
dataset_train['max_row'] = dataset_train.max(axis=1)
dataset_train['mean_row'] = dataset_train.mean(axis=1)
dataset_train['std_row'] = dataset_train.std(axis=1)

dataset_test1['min_row'] = dataset_test1.min(axis=1)
dataset_test1['max_row'] = dataset_test1.max(axis=1)
dataset_test1['mean_row'] = dataset_test1.mean(axis=1)
dataset_test1['std_row'] = dataset_test1.std(axis=1)

In [ ]:
dataset_test1['nan_count'] = dataset_test1.isnull().sum(axis=1)
#dataset_test1['nan_count']  = dataset_test1['nan_count']/dataset_test1['nan_count'].max()

In [ ]:
#dataset_test1['num_missing_std'] = dataset_test1.isna().std(axis=1).astype('float')
#dataset_test1['mean'] = dataset_test1.mean(axis=1)
#dataset_test1['median'] = dataset_test1.median(axis=1)
#dataset_test1['std'] = dataset_test1.std(axis=1)
#dataset_test1['mad'] = dataset_test1.mad(axis=1)
#dataset_test1['var'] = dataset_test1.var(axis=1)
#dataset_test1['skew'] = dataset_test1.skew(axis=1) 
#dataset_test1['max'] = dataset_test1.abs().max(axis=1)
#dataset_test1['min'] = dataset_test1.abs().min(axis=1) 

Below is the compiled list of the all feature engineering ideas tried out so far.<br>

X['num_missing'] = X.isna().sum(axis=1) # Number of NaN columns <br>
X['num_missing_std'] = X.isna().std(axis=1).astype('float') # Standard deviation of NaN columns count<br>
X['mean'] = X.mean(axis=1) # Mean of row values<br>
X['median'] = X.median(axis=1) # Median<br>
X['std'] = X.std(axis=1) # Standard Deviation<br>
X['mad'] = X.mad(axis=1) # Mean Absolute Deviation<br>
X['var'] = X.var(axis=1) # Variance<br>
X['skew'] = X.skew(axis=1) # Skewness<br>
X['max'] = X.abs().max(axis=1) # Max of row values<br>
X['min'] = X.abs().min(axis=1) # Min of row values<br>

In [ ]:
import tensorflow as tf

In [ ]:
# Detect TPU, return appropriate distribution strategy
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver() 
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    strategy = tf.distribute.get_strategy() 

print("REPLICAS: ", strategy.num_replicas_in_sync)

In [ ]:
from sklearn.preprocessing import QuantileTransformer, KBinsDiscretizer, PowerTransformer

In [ ]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy='median')
dataset_train = imputer.fit_transform(dataset_train)
dataset_test = imputer.transform(dataset_test1)

In [ ]:
#pt = PowerTransformer()
#dataset_train = pt.fit_transform(dataset_train)
#dataset_test = pt.transform(dataset_test)

In [ ]:
qt = QuantileTransformer(n_quantiles=100, output_distribution='uniform')
dataset_train = qt.fit_transform(dataset_train)
dataset_test = qt.transform(dataset_test)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
x_scaler = MinMaxScaler()
dataset_train_sc = x_scaler.fit_transform(dataset_train)
dataset_test_sc = x_scaler.transform(dataset_test)

In [ ]:
from tensorflow import keras
import tensorflow as tf
from tensorflow.keras.layers import Dense, Embedding
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Reshape
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model
#from tensorflow.keras.engine.input_layer import Input
from tensorflow.keras.layers import MaxPooling1D, GaussianNoise
from tensorflow.keras.layers import GlobalAveragePooling1D
from tensorflow.keras.layers import BatchNormalization

In [ ]:
from tensorflow.keras.constraints import max_norm

In [ ]:
def model_builder(lr):
    """Модель нейронной сети для агента"""
    inputA = keras.Input(shape=(119))
    line = GaussianNoise(0.000001)(inputA)
    line = Reshape((119,1))(line)
    
    #line = Conv1D(filters=8, kernel_size=2, padding='same', activation='relu')(line)
    #line = BatchNormalization()(line)
    #line = Dropout(0.2)(line)
    
    #line = MaxPooling1D(pool_size=2)(line)
    
    line = Conv1D(filters=16, kernel_size=2, activation='relu')(line)
    line = BatchNormalization()(line)
    line = Dropout(0.3)(line)
    
    #line = MaxPooling1D(pool_size=2)(line)

    line = Conv1D(filters=32, kernel_size=2, activation='relu')(line)
    line = BatchNormalization()(line)
    line = Dropout(0.5)(line)
    
    line = Conv1D(filters=64, kernel_size=3, activation='relu')(line)
    line = BatchNormalization()(line)
    line = Dropout(0.5)(line)
    
    #x = MaxPooling1D(pool_size=2)(line)
    #line = GlobalAveragePooling1D()(line)
    
    line = Flatten()(line)
    #line = Dense(512, activation='relu', kernel_regularizer=keras.regularizers.l1(0.000001))(line)
    #line = Dense(256, activation='relu', kernel_regularizer=keras.regularizers.l1(0.000001))(line)
    #line = Dropout(0.1)(line)
    #line = Dense(128, activation='relu')(line)
    #line = Dense(32, activation='relu')(line)
    #line = Dense(32, activation='relu',kernel_constraint=max_norm(2.),)(line)
    #line = Dense(16, activation='relu')(line)
    #line = Dense(64, activation='relu')(line)
    #line = Dense(32, activation='relu')(line)
    #line = Dense(16, activation='relu')(line)
    
    outputA = Dense(units=1, activation="sigmoid", kernel_regularizer=keras.regularizers.l1(0.01))(line)
    model = Model(inputs=inputA, outputs=outputA)
    #model = keras.models.load_model('models/model2')
    model.compile(loss = 'binary_crossentropy', optimizer = Adam(lr=lr), metrics=[tf.keras.metrics.AUC(name='auc'), 'binary_accuracy'],)
    return model

In [ ]:
lr=0.0001
with strategy.scope():
    model = model_builder(lr)

In [ ]:
model.summary()

In [ ]:
checkpoint_filepath = 'pseudo_best.h5'
save_model_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=False,
    monitor='val_auc',
    mode='max',
    verbose=1,
    save_best_only=True)

In [ ]:
#model = load_model('../input/tps0921-pre-nn/best_81332.h5')

repeated pseudo training

In [ ]:
#model = load_model('../input/tps0921-pre-nn/best_81204.h5')

In [ ]:
#preds = model.predict(dataset_test_sc)

In [ ]:
#for i in range(len(preds)):
#    preds[i] = 1 if preds[i] >= 0.5 else 0

In [ ]:
N_split = int(0.2 * len(dataset_train_sc))
dataset_sc_TRAIN = dataset_train_sc[:-N_split, :]
dataset_sc_VAL = dataset_train_sc[-N_split:, :]
y_TRAIN = y[:-N_split]
y_VAL = y[-N_split:]

In [ ]:
#import numpy as np
#big_dataset_X = np.concatenate([dataset_sc_TRAIN, dataset_test_sc], axis=0)
#len(big_dataset_X)

In [ ]:
#big_dataset_y = np.concatenate([y_TRAIN, preds], axis=0)
#len(big_dataset_y)

In [ ]:
#N = len(preds)/(len(preds)+len(y_TRAIN))
#N

In [ ]:
from tensorflow.keras.callbacks import ReduceLROnPlateau
reduce_lr = ReduceLROnPlateau(monitor='val_auc', factor=0.2, patience=10, min_lr=0.001, verbose=1, mode='max')

pseudo training

In [ ]:
EPOCHS = 50
#EPOCHS = 1
#model.fit(
#    big_dataset_X, big_dataset_y,
#    validation_data=(dataset_sc_VAL, y_VAL), epochs=EPOCHS, callbacks=[save_model_callback, reduce_lr], batch_size=2048, shuffle=True)

real training

In [ ]:
EPOCHS = 1000
#EPOCHS = 1
model.fit(
    dataset_sc_TRAIN, y_TRAIN,
    validation_data=(dataset_sc_VAL, y_VAL), epochs=EPOCHS, callbacks=[save_model_callback, reduce_lr], batch_size=1024, shuffle=True)

repeat few times 2 cells above

In [ ]:
model = load_model('pseudo_best.h5')

In [ ]:
preds = model.predict(dataset_test_sc)
output = pd.DataFrame({'Id': dataset_test1.index,'claim': preds[:,0]})
path = 'sample_submission.csv'
output.to_csv(path, index=False)
output 